In [49]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gp 
import plotly.express as px
import json
from matplotlib.colors import Normalize

In [5]:
#read shapefile
countryshp = gp.read_file(r"asset/mapdata202301070205/COUNTY_MOI_1090820.shp"
                         ,encoding='UTF-8')#without encoding is ok! 
# countryshp.index = countryshp['COUNTYNAME']
# 將shp檔案格式轉為geoJson檔案格式
countryshp.to_file('asset/mapdata202301070205/COUNTY_MOI_1090820.geojson', driver='GeoJSON', epsg=4326)  # 經緯度由epsg3826(TWD97)轉換為epsg4326(WGS84)

In [43]:
# 利用json模組，讀入geojson檔案：
# 讀取geojson
with open('asset/mapdata202301070205/COUNTY_MOI_1090820.geojson', encoding='utf8') as response:
    mapGeo = json.load(response)

# 觀察geojson資料狀況
print(mapGeo['features'][0]['properties'])
print(mapGeo['features'][1]['properties'])


{'COUNTYID': 'Z', 'COUNTYCODE': '09007', 'COUNTYNAME': '連江縣', 'COUNTYENG': 'Lienchiang County'}
{'COUNTYID': 'G', 'COUNTYCODE': '10002', 'COUNTYNAME': '宜蘭縣', 'COUNTYENG': 'Yilan County'}


In [10]:
# 制震宅資料整理
ALL_data = pd.read_excel("asset/ALL_data.xlsx")
ALL_data.drop("Unnamed: 0",axis=1,inplace=True)
selling_cases = len(ALL_data)
AST_data = ALL_data[ALL_data["制震宅"]]

In [47]:
AST_data["單價"] = pd.to_numeric(AST_data['單價'], errors='coerce')
AST_country_Price = AST_data[AST_data['單位'] == '萬/坪'].groupby('縣市')['單價'].agg('mean').reset_index()
AST_country_Price["縣市"] = AST_country_Price["縣市"].str.replace("台", "臺")
AST_country_Price["單價"] = round(AST_country_Price["單價"],1)

#此處依據geojson檔案內記載的鄉鎮市區清單資訊，建立一個Pandas表格，待會要將此表做為主表，把價格資料併進來：
# 依據圖資資料建立鄉鎮市區清單
country_df = pd.DataFrame.from_dict([i['properties'] for i in mapGeo['features']])
AST_country_Price_table = pd.merge(country_df, AST_country_Price, left_on='COUNTYNAME', right_on='縣市', how='left')
AST_country_Price_table

C:\Users\2293\AppData\Local\Temp\ipykernel_13256\741970274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AST_data["單價"] = pd.to_numeric(AST_data['單價'], errors='coerce')


,COUNTYID,COUNTYCODE,COUNTYNAME,COUNTYENG,縣市,單價
0,Z,09007,連江縣,Lienchiang County,NaN,NaN
1,G,10002,宜蘭縣,Yilan County,宜蘭縣,29.1
2,N,10007,彰化縣,Changhua County,彰化縣,27.0
3,M,10008,南投縣,Nantou County,NaN,NaN
4,P,10009,雲林縣,Yunlin County,雲林縣,27.4
5,C,10017,基隆市,Keelung City,NaN,NaN
6,A,63000,臺北市,Taipei City,臺北市,137.2
7,F,65000,新北市,New Taipei City,新北市,65.6
8,B,66000,臺中市,Taichung City,臺中市,43.5
9,D,67000,臺南市,Tainan City,臺南市,41.3


In [51]:
# 繪製地坪單價地圖
fig = px.choropleth_mapbox(AST_country_Price_table,                                                # 資料表
                           geojson=mapGeo,                                    # 地圖資訊
                           locations='COUNTYCODE',                           # df要對應geojson的id名稱
                           featureidkey='properties.COUNTYCODE',             # geojson對應df的id名稱
                           color='單價',                                      # 顏色區分對象
                           color_continuous_scale='Viridis',                  # 設定呈現的顏色
                           range_color=(round(np.nanmin(AST_country_Price_table['單價'])),    # 顏色的值域範圍
                                        round(np.nanmax(AST_country_Price_table['單價']))),   
                           mapbox_style='carto-positron',                    # mapbox地圖格式
                           zoom=6,                                           # 地圖縮放大小: 數字愈大放大程度愈大
                           center={'lat': 23.5832, 'lon': 120.5825},         # 地圖中心位置: 此處設定台灣地理中心碑經緯度
                           opacity=0.5,                                               # 設定顏色區塊的透明度 數值愈大愈不透明
                           hover_data=['縣市', "單價"]  # 設定游標指向資訊
                          )
fig.update_layout(margin={'r':0, 't':0, 'l':0, 'b':0})
fig.show()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed